In [2]:
# from flask_other.app import app
from flask_socketio import disconnect as dc
from flask import Flask
from flask_socketio import SocketIO, emit,join_room,leave_room
from flask_socketio import disconnect
from threading import Lock
import os,sys
# sys.path.append('../../../../Chatbot1.0/')
# sys.path.append('/home/kai/data/Chatbot1.0/API/v1/frontend/')
app=Flask("demo")
from flask import  render_template,request
import time
import re

import pandas as pd
import numpy as np
import time
import re
import requests



##################################
async_mode = None
app = Flask(__name__)
app.config['SECRET_KEY'] = 'secret!'
socketio = SocketIO(app, async_mode=async_mode)

thread = None
thread_lock = Lock()
clients = []
sentences = []

name_space = '/chat'

@app.route('/')
def index():
    return render_template('index.html', async_mode=socketio.async_mode)


@socketio.on('client_send',namespace=name_space)
def client_msg(msg):  
    uid = request.sid
    sentence = msg.get('data')
    sentence = decode(sentence)
    sentences.append(sentence)
    print(sentence)
    response = cache.chat(uid, sentence)
    if response == 'end':
        socketio.emit('my_response',{'data':'感谢选择江苏逸能，再见！'},room = uid, namespace=name_space)
        disconnect_frontend(uid)
    elif response is None:
        socketio.emit('my_response',{'data':'当前会话已经过期，感谢选择江苏逸能，再见！'},room = uid, namespace=name_space)
        disconnect_frontend(uid)
    else:
        socketio.emit('my_response',{'data':response},room = uid, namespace=name_space)

def decode(msg):
    msg = re.sub(r'%u', r'\u', msg)
    msg = msg.encode('latin-1').decode('unicode_escape')
    return msg

#sned message to a specific user
def unique_message(uid):
    socketio.emit('unique_messgae',{'data':'are you still here'}, room=uid, namespace=name_space)
    
def disconnect_frontend(uid):
    socketio.emit('my_response',{'status':'disconnected'},room = uid, namespace=name_space)


@socketio.on('disconnect',namespace=name_space)
def disconnect():
    uid = request.sid
    cache.remove_session(uid)
    leave_room(uid)
    dc()
    print('{} is disconnected'.format(uid))
    
@socketio.on('connect',namespace=name_space)
def connect():
    print('connect')
    print(request.sid)
    uid = request.sid
    clients.append(uid)
    print(len(clients))
    
    join_room(uid)
    if cache.create_session(uid):
        response = cache.chat(uid, '')
        if response is not None:
            socketio.emit('my_response',{'data':response},room = uid,namespace=name_space) #the first sentence
            return None
        
    else:
        socketio.emit('my_response',{'data':'server busy. please click new conv'},room = uid,namespace=name_space) 
        disconnect_frontend(uid)
    



In [3]:
sentences[0]

IndexError: list index out of range

In [5]:
decode(sentences[0])

'是的'

In [ ]:
print('http://10.0.24.31:8899/')
print('http://0.0.0.0:6006/')
socketio.run(app,'0.0.0.0',6006)

http://10.0.24.31:8899/
http://0.0.0.0:6006/
connect
77a9ee58cf5d44c6a624fb259ed93533
1


Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache


是的


Loading model cost 0.840 seconds.
Prefix dict has been built succesfully.


我破产了
label received is 0
我真的没有钱
label received is 1
jump action is triggered
马上就还
是
session 77a9ee58cf5d44c6a624fb259ed93533 is removed
77a9ee58cf5d44c6a624fb259ed93533 is disconnected
77a9ee58cf5d44c6a624fb259ed93533 is disconnected


In [1]:
import time
import gc
class Cache:
    def __init__(self, model_dict,max_session=1000):
        self.max_session = 1000
        self.inform_interval = 30
        #{'uid': {'stragety': Tree(), 'time_response': <time>, 'time_inform': <>}
        self.active_session = {}
        self.model_dict = model_dict
        
        
    def create_session(self, uid):
        if len(self.active_session) < self.max_session:
            self.active_session[uid] = {}
            self.active_session[uid].update({'stragety':TreeStage1()})
            self.active_session[uid].update({'time_response':time.time()})
            self.active_session[uid].update({'time_inform':time.time()})
            return True
        else:
            return False
        
    def remove_session(self,uid):
        try:
            del self.active_session[uid]
            print('session {} is removed'.format(uid))
        except KeyError:
                pass
        gc.collect()
            
    def chat(self,uid,sentence):
        if self.active_session.get(uid) is not None:
            response = self.active_session[uid]['stragety'].process(sentence, self.model_dict)
        else:
            response = None
        return response
        
    
    
    def purge_inactive(self):
        sec=500
        current = time.time()
        inactive_list = []
        for each in self.session_timer:
            if current - self.session_timer[each] > sec:
                print('{} session is inactive, will be removed!'.format(each))
                inactive_list.append(each)
        for key in inactive_list:
            try:
                del self.active_session[key]
            except KeyError:
                pass
            try:
                del self.session_timer[key]
            except KeyError:
                pass
            print('{} is removed'.format(key))
        gc.collect()
        
    def inform_inactive(self):
        pass
        
        
        
import sys,os
sys.path.append('../../../MLModel/code/OneClickTraining/')
sys.path.append('../../../MLModel/code/TreeModel/')
from all_model_py import *
import pickle
from chatbotv1 import *



models_list = ['IDClassifier','CutDebt','IfKnowDebtor','WillingToPay','Installment','ConfirmLoan']
savedModel_path = '../../../MLModel/savedModel/{}/{}.pickle'
model_dict = {}
for each_model in models_list:
    model_dict[each_model] = pickle.load(open(savedModel_path.format(each_model,each_model), 'rb'))
    
model_dict['OtherClassifier'] = OtherClassifier()
model_dict['StopClassifier'] = StopClassifier()
model_dict['InitClassifier'] = InitClassifier()    

cache = Cache(model_dict)

In [ ]:
import sys,os
sys.path.append('../../../MLModel/code/OneClickTraining/')
sys.path.append('../../../MLModel/code/TreeModel/')
from all_model_py import *
import pickle
from chatbotv1 import *



models_list = ['IDClassifier','CutDebt','IfKnowDebtor','WillingToPay','Installment','ConfirmLoan']
savedModel_path = '../../../MLModel/savedModel/{}/{}.pickle'

model_dict = {}
for each_model in models_list:
    model_dict[each_model] = pickle.load(open(savedModel_path.format(each_model,each_model), 'rb'))
    
model_dict['OtherClassifier'] = OtherClassifier()
model_dict['StopClassifier'] = StopClassifier()
model_dict['InitClassifier'] = InitClassifier()    